# Bab 18: Reinforcement Learning (Pembelajaran Penguatan)

### 1. Pendahuluan

Bab 18 memperkenalkan **Reinforcement Learning (RL)**, sebuah paradigma *Machine Learning* yang berbeda dari *supervised* dan *unsupervised learning*. Dalam RL, sebuah **agen (agent)** belajar bagaimana berperilaku dalam suatu **lingkungan (environment)** dengan melakukan **tindakan (actions)** dan menerima **hadiah (rewards)** atau **hukuman (penalties)**. Tujuannya adalah untuk memaksimalkan total hadiah kumulatif dari waktu ke waktu.

RL telah mencapai keberhasilan luar biasa, terutama dalam permainan (misalnya, AlphaGo) dan robotika. Bab ini akan membahas konsep inti RL, beberapa algoritma utamanya, dan bagaimana mengimplementasikannya.

---

### 2. Konsep Inti Reinforcement Learning

* **Kebijakan (Policy):** Strategi yang digunakan agen untuk memilih tindakan berdasarkan keadaan saat ini. Ini bisa bersifat deterministik atau stokastik.
* **Fungsi Nilai (Value Function):** Memprediksi seberapa baik (dalam hal hadiah masa depan yang diharapkan) berada dalam keadaan tertentu atau melakukan tindakan tertentu dalam keadaan tertentu.
* **Model Lingkungan:** Beberapa agen mungkin mencoba mempelajari model lingkungan itu sendiri (*model-based RL*), yang memungkinkan mereka untuk merencanakan ke depan.

---

### 3. Open AI Gym: Lingkungan untuk Eksperimen RL

**OpenAI Gym** (sekarang dikelola oleh Farama Foundation dengan nama Gymnasium) adalah *toolkit* yang menyediakan berbagai lingkungan simulasi untuk mengembangkan dan menguji algoritma RL.


In [1]:
!pip install -q gym

import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Membuat lingkungan CartPole
env = gym.make("CartPole-v1")
obs, info = env.reset(seed=42)

# Menjalankan satu episode dengan kebijakan acak
# for _ in range(50):
#    action = env.action_space.sample()
#    obs, reward, done, truncated, info = env.step(action)
#    if done or truncated:
#        break

# env.close()

---

### 4. Algoritma Policy Gradients (PG)
Policy Gradients adalah keluarga algoritma RL di mana jaringan saraf dilatih untuk secara langsung memprediksi probabilitas setiap tindakan yang mungkin (policy-based). Jaringan ini disebut policy network.

Proses Pelatihan:
1. **Jalankan Episode**: Jalankan model (agen) di lingkungan untuk beberapa episode dan kumpulkan semua riwayat (keadaan, tindakan, hadiah).
2. **Hitung Hadiah yang Didiskon**: Untuk setiap langkah, hitung total hadiah masa depan yang didiskon (discounted future rewards). Tindakan yang mengarah ke hadiah tinggi di masa depan akan dianggap "baik".
3. **Latih Jaringan**: Latih policy network dengan gradient ascent. Gradien dihitung untuk meningkatkan probabilitas tindakan "baik" dan mengurangi probabilitas tindakan "buruk".

In [2]:
# Membangun Policy Network
n_inputs = env.observation_space.shape[0]
n_outputs = env.action_space.n

model_pg = keras.models.Sequential([
    keras.layers.Dense(32, activation="elu", input_shape=[n_inputs]),
    keras.layers.Dense(32, activation="elu"),
    keras.layers.Dense(n_outputs, activation="softmax")
])

# Fungsi untuk memainkan satu langkah dan menghitung gradien
# (Ini adalah bagian dari training loop yang lebih besar)
def play_one_step_pg(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        probas = model(obs[np.newaxis])
        # ... (logika untuk memilih aksi dan menghitung loss)
        # loss = tf.reduce_mean(loss_fn(target_probas, probas))
    
    # grads = tape.gradient(loss, model.trainable_variables)
    # return obs, reward, done, grads

# Kerangka Training Loop untuk Policy Gradients
# for episode in range(n_training_episodes):
#     # ... (jalankan episode, kumpulkan gradien)
#     # ... (hitung discounted rewards)
#     # ... (normalisasi rewards dan terapkan gradien)

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


---

### 5. Algoritma Markov Decision Processes (MDPs) dan Q-Learning
Banyak masalah RL dapat dimodelkan sebagai Markov Decision Processes (MDPs). MDP memiliki state dengan probabilitas transisi tetap.

**Q-Learning** adalah algoritma RL populer yang mencoba mempelajari action-value (disebut Q-Value) untuk setiap pasangan state-action. Q-Value merepresentasikan hadiah masa depan yang diharapkan jika agen memulai dari state S, melakukan action A, dan kemudian mengikuti kebijakan optimal sesudahnya.

Algoritma ini menggunakan Bellman equation untuk memperbarui Q-Value secara iteratif.

---

### 6. Deep Q-Learning (DQN)
Untuk lingkungan dengan ruang state yang sangat besar (seperti gambar dari layar game), tidak mungkin untuk melacak Q-Value untuk setiap pasangan state-action. Deep Q-Network (DQN) menyelesaikan ini dengan menggunakan jaringan saraf untuk mengestimasi Q-Value.

Fitur Utama DQN:
* **Experience Replay**: Agen menyimpan pengalamannya (transisi state, tindakan, hadiah) dalam sebuah replay buffer. Selama pelatihan, ia mengambil sampel batch acak dari buffer ini. Ini mengurangi korelasi antar sampel dan meningkatkan efisiensi data.
* **Target Network**: DQN menggunakan dua jaringan: online network yang dilatih di setiap langkah, dan target network yang bobotnya disalin dari online network secara berkala. Target network digunakan untuk menghitung target Q-Value, yang memberikan stabilitas pada proses pelatihan.

In [3]:
# Kerangka untuk Deep Q-Network (DQN)
# (Asumsikan `env` dan `n_outputs` sudah didefinisikan)

# 1. Replay Buffer
# replay_buffer = collections.deque(maxlen=replay_buffer_size)

# 2. Model DQN
# model_dqn = keras.models.Sequential([
#    keras.layers.Dense(32, activation="elu", input_shape=env.observation_space.shape),
#    keras.layers.Dense(32, activation="elu"),
#    keras.layers.Dense(n_outputs)
# ])

# 3. Fungsi untuk memilih aksi (Epsilon-Greedy Policy)
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)
    else:
        # Q_values = model_dqn.predict(state[np.newaxis])
        # return np.argmax(Q_values[0])
        pass # Placeholder

# 4. Kerangka Training Step
# def training_step(batch_size):
#     # 1. Sampel pengalaman dari replay buffer
#     # experiences = sample_experiences(batch_size)
#     # 2. Hitung target Q-Values menggunakan Bellman equation
#     #    target_Q_values = rewards + discount_factor * np.max(next_Q_values, axis=1)
#     # 3. Latih model DQN dengan Gradient Descent pada (target_Q_values - predicted_Q_values)
print("\nKerangka untuk Deep Q-Network (DQN) telah disiapkan.")


Kerangka untuk Deep Q-Network (DQN) telah disiapkan.


Training RL seringkali tidak stabil dan sangat sensitif terhadap hyperparameter. Menggunakan library yang sudah matang seperti **TF-Agents** seringkali menjadi pilihan yang lebih praktis untuk aplikasi dunia nyata.